<a href="https://colab.research.google.com/github/matehr/stockmarket/blob/master/StockResearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install html5lib==0.9999999
!git clone https://github.com/rahulrrixe/SEC-Edgar.git
%cd SEC-Edgar
!python setup.py install

#!pip install SECEdgar
#import SECEdgar

fatal: destination path 'SEC-Edgar' already exists and is not an empty directory.
/content/SEC-Edgar
running install
running bdist_egg
running egg_info
writing SECEdgar.egg-info/PKG-INFO
writing dependency_links to SECEdgar.egg-info/dependency_links.txt
writing entry points to SECEdgar.egg-info/entry_points.txt
writing requirements to SECEdgar.egg-info/requires.txt
writing top-level names to SECEdgar.egg-info/top_level.txt
reading manifest template 'MANIFEST.in'
writing manifest file 'SECEdgar.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/SECEdgar
copying build/lib/SECEdgar/__init__.py -> build/bdist.linux-x86_64/egg/SECEdgar
copying build/lib/SECEdgar/crawler.py -> build/bdist.linux-x86_64/egg/SECEdgar
creating build/bdist.linux-x86_64/egg/SECEdgar/filings
copying build/lib/SECEdgar/filings/__init__.py -> build/bdist.linux-x86_64/egg/SECEdgar

In [7]:
import re, requests

def getCIKs(TICKERS):
    URL = 'http://www.sec.gov/cgi-bin/browse-edgar?CIK={}&Find=Search&owner=exclude&action=getcompany'
    CIK_RE = re.compile(r'.*CIK=(\d{10}).*')    
    cik_dict = {}
    for ticker in TICKERS:
        f = requests.get(URL.format(ticker), stream = True)
        results = CIK_RE.findall(f.text)
        if len(results):
            results[0] = int(re.sub('\.[0]*', '.', results[0]))
            cik_dict[str(ticker).upper()] = str(results[0])
    return cik_dict


cik_dict = getCIKs(['amzn'])
print(cik_dict)

{'AMZN': '1018724'}


In [6]:
from SECEdgar.filings import Filing, FilingType
my_filings = Filing(cik=cik_dict['AMZN'], filing_type=FilingType.FILING_10K)
print(my_filings.get_urls())

['https://www.sec.gov/Archives/edgar/data/1018724/000101872420000004/0001018724-20-000004.txt', 'https://www.sec.gov/Archives/edgar/data/1018724/000101872419000004/0001018724-19-000004.txt', 'https://www.sec.gov/Archives/edgar/data/1018724/000101872418000005/0001018724-18-000005.txt', 'https://www.sec.gov/Archives/edgar/data/1018724/000101872417000011/0001018724-17-000011.txt', 'https://www.sec.gov/Archives/edgar/data/1018724/000101872416000172/0001018724-16-000172.txt', 'https://www.sec.gov/Archives/edgar/data/1018724/000101872415000006/0001018724-15-000006.txt', 'https://www.sec.gov/Archives/edgar/data/1018724/000101872414000006/0001018724-14-000006.txt', 'https://www.sec.gov/Archives/edgar/data/1018724/000119312513028520/0001193125-13-028520.txt', 'https://www.sec.gov/Archives/edgar/data/1018724/000119312512032846/0001193125-12-032846.txt', 'https://www.sec.gov/Archives/edgar/data/1018724/000119312511050136/0001193125-11-050136.txt']


In [11]:
def retrieve_10K(cik_dict):
    endpoint = r"https://www.sec.gov/cgi-bin/browse-edgar"

    for cik in cik_dict.values():
        param_dict = {
            'action': 'getcompany',
            'CIK': cik,
            'type': '10-k',
            'datea': '20091101',
            'owner': 'exclude',
            'start': '',
            'output': 'atom',
            'count': '1'
        }

        response = requests.get(url=endpoint, params=param_dict)
        return response.content
        
content = retrieve_10K(cik_dict)
content

b'<?xml version="1.0" encoding="ISO-8859-1" ?>\n  <feed xmlns="http://www.w3.org/2005/Atom">\n    <author>\n      <email>webmaster@sec.gov</email>\n      <name>Webmaster</name>\n    </author>\n    <company-info>\n      <addresses>\n        <address type="mailing">\n          <city>SEATTLE</city>\n          <state>WA</state>\n          <street1>410 TERRY AVENUE NORTH</street1>\n          <zip>98109</zip>\n        </address>\n        <address type="business">\n          <city>SEATTLE</city>\n          <phone>2062661000</phone>\n          <state>WA</state>\n          <street1>410 TERRY AVENUE NORTH</street1>\n          <zip>98109</zip>\n        </address>\n      </addresses>\n      <assigned-sic>5961</assigned-sic>\n      <assigned-sic-desc>RETAIL-CATALOG &amp;amp; MAIL-ORDER HOUSES</assigned-sic-desc>\n      <assigned-sic-href>https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&amp;SIC=5961&amp;owner=exclude&amp;count=10</assigned-sic-href>\n      <cik>0001018724</cik>\n      <cik